<h1>Chapter 2 - Tokens and Token Embeddings</h1>
<i>Exploring tokens and embeddings as an integral part of building LLMs</i>


<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter02/Chapter%202%20-%20Tokens%20and%20Token%20Embeddings.ipynb)

---

This notebook is for Chapter 2 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>


### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [ ]:
# %%capture
# !pip install --upgrade transformers==4.41.2 sentence-transformers==3.0.1 gensim==4.3.2 scikit-learn==1.5.0 accelerate==0.31.0 peft==0.11.1 scipy==1.10.1 numpy==1.26.4

# Downloading and Running An LLM

The first step is to load our model onto the GPU for faster inference. Note that we load the model and tokenizer separately and keep them as such so that we can explore them separately.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: Heartfelt Apologies for the Gardening Mishap


Dear


In [3]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001]], device='cuda:0')


In [4]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|assistant|>


In [5]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001,  3323,   622, 29901, 17778, 29888,  2152,
          6225, 11763,   363,   278, 19906,   292,   341,   728,   481,    13,
            13,    13, 29928,   799]], device='cuda:0')

In [6]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


# Comparing Trained LLM Tokenizers


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    print(token_ids)
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [8]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [9]:
show_tokens(text, "bert-base-uncased")

[101, 2394, 1998, 3007, 3989, 100, 100, 2265, 1035, 19204, 2015, 6270, 3904, 12005, 2546, 1027, 1027, 1028, 1027, 2842, 1024, 2048, 21628, 2015, 1024, 1000, 1000, 2093, 21628, 2015, 1024, 1000, 1000, 2260, 1012, 1014, 1008, 2753, 1027, 5174, 102]
[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [10]:
show_tokens(text, "bert-base-cased")

[101, 1483, 1105, 8784, 23203, 9159, 2162, 2240, 5301, 13821, 24805, 100, 100, 1437, 168, 22559, 1116, 143, 7264, 1162, 7330, 8468, 8914, 134, 134, 135, 134, 1950, 131, 1160, 27629, 4832, 131, 107, 107, 2677, 27629, 4832, 131, 107, 107, 1367, 119, 121, 115, 1851, 134, 4372, 102]
[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [11]:
show_tokens(text, "gpt2")

[198, 15823, 290, 20176, 40579, 14887, 6234, 198, 8582, 236, 113, 16268, 116, 253, 198, 12860, 62, 83, 482, 641, 10352, 6045, 1288, 361, 6624, 18189, 2073, 25, 734, 22524, 11097, 220, 220, 220, 366, 7683, 22524, 25, 366, 220, 220, 220, 220, 220, 220, 366, 198, 1065, 13, 15, 9, 1120, 28, 8054, 198]

 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [12]:
show_tokens(text, "google/flan-t5-small")

[1566, 11, 3087, 4111, 16359, 20091, 8015, 3, 2, 3, 2, 504, 834, 235, 2217, 7, 10747, 7, 15, 14794, 3, 15, 40, 99, 3274, 2423, 2490, 2423, 1307, 10, 192, 3808, 7, 10, 121, 96, 5245, 3808, 7, 10, 96, 96, 8013, 632, 1935, 1752, 2423, 6007, 3, 1]
English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [13]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")

[198, 23392, 323, 78387, 50125, 198, 9468, 236, 113, 18630, 116, 253, 198, 3528, 29938, 3641, 2290, 4508, 624, 2669, 775, 25, 1403, 23204, 3047, 262, 330, 14853, 23204, 25, 330, 996, 6360, 717, 13, 15, 9, 1135, 28, 5067, 198]

 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [14]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")

[222, 23543, 480, 38657, 26965, 222, 3822, 260, 151, 244, 32589, 277, 222, 2276, 100, 8433, 3208, 1686, 4378, 630, 2394, 832, 63, 3161, 16992, 1941, 283, 332, 28033, 16992, 63, 332, 981, 332, 222, 54, 55, 51, 53, 47, 58, 53, 66, 59, 53, 53, 222]

 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [15]:
show_tokens(text, "facebook/galactica-1.3b")

[221, 28003, 312, 18320, 36247, 29196, 221, 195, 276, 259, 136, 43089, 139, 276, 221, 14592, 85, 24948, 12051, 4830, 14155, 243, 1414, 243, 52, 51, 3683, 48, 682, 279, 6510, 48, 24, 344, 24, 5753, 279, 6510, 48, 243, 24, 863, 24, 221, 39, 40, 36, 38, 32, 43, 38, 51, 44, 38, 38, 221]

 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [16]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

[29871, 13, 24636, 322, 315, 3301, 1806, 1964, 26664, 8098, 13, 243, 162, 145, 184, 29871, 236, 187, 162, 13, 4294, 29918, 517, 12360, 7700, 6213, 25342, 1275, 6736, 1683, 29901, 1023, 18859, 6160, 1678, 376, 12753, 18859, 29901, 376, 539, 376, 13, 29896, 29906, 29889, 29900, 29930, 29945, 29900, 29922, 29953, 29900, 29900, 13]
 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

# Contextualized Word Embeddings From a Language Model (Like BERT)

In [17]:
from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

In [18]:
output.shape

torch.Size([1, 4, 384])

In [19]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
 world
[SEP]


In [20]:
output

tensor([[[-3.4816,  0.0861, -0.1819,  ..., -0.0612, -0.3911,  0.3017],
         [ 0.1898,  0.3208, -0.2315,  ...,  0.3714,  0.2478,  0.8048],
         [ 0.2071,  0.5036, -0.0485,  ...,  1.2175, -0.2292,  0.8582],
         [-3.4278,  0.0645, -0.1427,  ...,  0.0658, -0.4367,  0.3834]]],
       grad_fn=<NativeLayerNormBackward0>)

# Text Embeddings (For Sentences and Whole Documents)

In [21]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to text embeddings
vector = model.encode("Best movie ever!")

In [22]:
vector.shape

(768,)

In [34]:
vector

array([-2.02203933e-02,  4.57696877e-02, -1.26637146e-02, -3.37991631e-03,
       -2.54910300e-03, -3.31096235e-03, -4.58366945e-02,  2.32390799e-02,
       -3.12585123e-02, -3.15588824e-02, -2.19783355e-02,  3.67821753e-03,
        3.39105981e-03, -1.88130587e-02, -2.65821870e-02, -1.45892315e-02,
        3.20066400e-02, -3.66493082e-03,  1.75410733e-02,  5.43209612e-02,
       -3.96661647e-02,  9.93637647e-03, -3.27205285e-02, -1.62909944e-02,
        6.36525359e-03, -1.24459760e-02,  1.04737049e-02,  3.08674313e-02,
        7.47453189e-03, -3.86319868e-02, -6.19924963e-02, -3.25308628e-02,
       -1.40268244e-02,  5.16586639e-02,  1.69651707e-06, -1.74092929e-04,
       -1.27881172e-03, -4.15052772e-02,  5.00347884e-03,  4.58600894e-02,
       -2.85337027e-02,  5.02370670e-02, -1.75183658e-02, -3.98830482e-04,
        1.63311344e-02,  5.89936748e-02, -2.21730098e-02, -4.00975086e-02,
       -6.60439581e-03, -2.97632981e-02, -3.18221636e-02, -1.77945811e-02,
       -1.54363075e-02, -

In [36]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 re

In [23]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [28]:
model.most_similar([model['apples']], topn=11)

[('apples', 1.0),
 ('peaches', 0.8623535633087158),
 ('oranges', 0.8594476580619812),
 ('cherries', 0.8461859226226807),
 ('mangoes', 0.8264981508255005),
 ('apricots', 0.8242633938789368),
 ('strawberries', 0.8229067921638489),
 ('potatoes', 0.8179376721382141),
 ('melons', 0.7980057597160339),
 ('berries', 0.7946050763130188),
 ('vegetables', 0.792052149772644)]

# Recommending songs by embeddings

In [29]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [30]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [31]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [32]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('6658', 0.9975031018257141),
 ('2704', 0.9970973134040833),
 ('11502', 0.9970895648002625),
 ('3167', 0.9970822930335999),
 ('5549', 0.9967572689056396),
 ('2849', 0.9962003827095032),
 ('5586', 0.9961282014846802),
 ('3136', 0.996070146560669),
 ('2640', 0.9959730505943298),
 ('2886', 0.9959593415260315)]

In [33]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [34]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
6658,(Bang Your Head) Metal Health,Quiet Riot
2704,Over The Mountain,Ozzy Osbourne
11502,Turbo Lover,Judas Priest
3167,Unchained,Van Halen
5549,November Rain,Guns N' Roses


In [35]:
print_recommendations(2172)

,title,artist
id,,
6658,(Bang Your Head) Metal Health,Quiet Riot
2704,Over The Mountain,Ozzy Osbourne
11502,Turbo Lover,Judas Priest
3167,Unchained,Van Halen
5549,November Rain,Guns N' Roses


In [36]:
print_recommendations(842)

,title,artist
id,,
27081,"Give Me Everything (w\/ Ne-Yo, Afrojack & Nayer)",Pitbull
5788,Drop It Like It's Hot (w\/ Pharrell),Snoop Dogg
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
196,I'll Be Missing You,Puff Daddy & The Family
886,Heartless,Kanye West
